# Creating "All grid points of field" from the Meteorological Predictor Fields as Input for RF-based ML-Models
Version 18 January 2024, Selina Kiefer

### Input: csv-files
continuous timeseries of meteorological predictors as 2d-fields in csv-format
### Output: csv-file
continuous timeseries of all grid points of the meteorological predictors as a long vector per date in csv-format

#### Define the paths' and files' names

In [ ]:
# Set the needed path and file names.
PATH_defined_functions = './Defined_Functions/'

PATH_input_data = './Data_in_csv_Format/'
ifiles_input_data = ['era5_u10_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_z100_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                     'era5_z250_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                    'era5_z500_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                    'era5_z850_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                    'era5_t850_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                    'era5_H850_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                    'era5_u300_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv',
                    'era5_msl_60W_60E_20N_80N_1950_2020_only_Oct_Apr_lead_time_14d.csv']

PATH_output_file = './Data_in_csv_Format/'
file_name_output_file = 'era5_u10_z100_z250_z500_z850_t850_H850_u300_msl_60W_60E_20N_80N_1950_2020_lead_time_14d.csv'

#### Import the necessary packages and functions

In [ ]:
# Import the necessary python packages.
import yaml
import pandas as pd
import numpy as np

In [ ]:
# Import the necessary functions.
import sys
sys.path.insert(1,PATH_defined_functions)
from read_in_csv_data import *

#### List the predictors to be combined

In [ ]:
# List the desired predictors and set how many of these should be taken from the first 
# dataframe. From all other dataframes, only 1 predictor is taken (if more are needed, list
# these input files multiple times in "ifiles_input_data"). It is necessary to take the time as
# a predictor since the data will be grouped by date later.
desired_predictors = ['time', 'latitude', 'longitude', 'month', 'u10', 'z100', 'z250', 'z500', 'z850', 't850', 'H850', 'u300', 'msl']
number_of_predictors_in_first_dataframe = 5
time_column_name = 'time'

#### Combine all predictors into one dataframe

In [ ]:
# A new dataframe is created and the desired predictors from the first data file are written
# into it.
df_combined_input_data = pd.DataFrame()
df_input_data = read_in_csv_data(PATH_input_data, ifiles_input_data[0])
for i in range(number_of_predictors_in_first_dataframe):
    df_combined_input_data[desired_predictors[i]] = df_input_data [desired_predictors[i]]

In [ ]:
# From all other dataframes, the specified predictor is added to this new dataframe.
for k in range(len(ifiles_input_data)-1):
    df_input_data = read_in_csv_data(PATH_input_data, ifiles_input_data[k+1])
    df_combined_input_data[desired_predictors[i+k+1]] = df_input_data [desired_predictors[i+k+1]]

In [ ]:
# Now the time is set as the index and the data is grouped by date.
df_combined_input_data[time_column_name] = pd.to_datetime(df_combined_input_data[time_column_name])
df_combined_input_data = df_combined_input_data.set_index(time_column_name)
ds_input_data_grouped = df_combined_input_data.groupby([df_combined_input_data.index.year, df_combined_input_data.index.month, df_combined_input_data.index.day], as_index=False)

In [ ]:
# The so grouped data is converted into a pandas dataframe.
df_input_data_grouped = pd.DataFrame(ds_input_data_grouped)

In [ ]:
# Since the data is stacked, only the relevant column containing all the predictors is taken.
df_input_data_grouped = df_input_data_grouped[1]

#### Reshape the data so it can be used as input for RF-based ML-models directly

In [ ]:
# In a next step the data is reshaped so that the data can be used directly with the machine 
# learning model later. Therefore, it needs to have in one dimension the same length as the 
# ground truth data. Here, this is the time. So for every date, all predictors are put into a 
# separate row and then appended to a list. The predictors are thereby sorted in the same way
# in each row.
reshaped_data = []
for l in range(len(df_input_data_grouped)):
    single_day = np.array(df_input_data_grouped[l])
    single_day = single_day.reshape(1,-1)
    reshaped_data.append(single_day)

In [ ]:
# The so created list containing all the predictors is converted into a pandas dataframe 
# again.
df_reshaped_data = pd.DataFrame(np.squeeze(reshaped_data))

#### Add the time information again to the reshaped data

In [ ]:
# Since the time got lost by using .groupby(), a separate new dataframe is created containing
# only the time. To this dataframe, three new columns are added containing the year, month and 
# day.
df_combined_input_data = df_combined_input_data.reset_index()
df_time = pd.DataFrame()
df_time[time_column_name] = pd.to_datetime(df_combined_input_data[time_column_name])
df_time = df_time.set_index(time_column_name)
df_time['year'] = df_time.index.year
df_time['month'] = df_time.index.month
df_time['day'] = df_time.index.day
df_time = df_time.reset_index()

In [ ]:
# This new dataframe is then grouped by date and 'averaged' resulting in a daily time-
# series but separated into year, month and day.
df_time = df_time.set_index(time_column_name)
ds_time_mean = df_time.groupby([df_time.index.year, df_time.index.month, df_time.index.day], as_index=False).mean().astype(int).astype(str) 
df_time_mean = pd.DataFrame(ds_time_mean)

In [ ]:
# The separated timeseries is now combined into a single daily timeseries (nothing needs to be
# changed here).str
daily_timeseries = (df_time_mean['year'].astype(str)+'-'+df_time_mean['month'].astype(str)+'-'+df_time_mean['day']).astype(str)

In [ ]:
# In the next step, the time is added to the dataframe containing the statistics as predictors 
# (nothing needs to be changed here).
df_reshaped_data.insert(0, time_column_name, daily_timeseries)

#### Doublecheck the representation of the data

In [ ]:
# Check if everything is reshaped and sorted correctly.
df_reshaped_data.head()

In [ ]:
# Also check if everything is sorted, renamed or removed correctly at the end of the
# dataframe.
df_reshaped_data.tail()

#### Save the data in csv format

In [ ]:
# Save the pandas dataframe in csv-format.
df_reshaped_data.to_csv(PATH_output_file+file_name_output_file)

In [ ]:
# End of Program